In [1]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.service import Service
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from time import sleep
from bs4 import BeautifulSoup
import pandas as pd
import datetime
import csv
import re

url = 'https://www.thestar.com.my/search?query=bursa'
options = Options()
options.headless = True
options.add_argument('--no-sandbox')
driver = webdriver.Firefox(service=Service(GeckoDriverManager().install()), options=options)
driver.get(url)

C:\Users\irwin\AppData\Local\Temp\ipykernel_19260\1611968036.py:18: DeprecationWarning: headless property is deprecated, instead use add_argument('-headless')
  options.headless = True


In [2]:
def get_soup(driver): 
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    return soup

articlelist = []

# Function to scrape the articles from webpage
def get_articles(soup):
    i=0
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH,"//div[@class='queryly_item_container']")))
    articles = driver.find_elements(By.XPATH,"//div[@class='queryly_item_container']")
    
    print(f'Total articles to process: {len(articles)}')
    try:    
        for item in articles:
            i+=1
            print(f'Extracting articles {i} ...')
            article = item.find_element(By.XPATH,".//h2[@class='f18']")
            #article = item.find('h2', class_ = "f18")
            article = {
                'Title': article.text.strip() if article is not None else '',
                'Date': convert(item.find_element(By.XPATH,".//span[@class='timestamp']").text)
                }
            articlelist.append(article)
    except Exception as e:
        print(e)
        pass

# Function to convert string to datetime
def convert(date_str):
    format = '%d%b%Y'  # The format
    date = date_str.split()
    
    datetime_str = datetime.datetime.strptime(date[0] + date[1][0:3] + date[2], format).strftime("%d/%m/%Y")
   
    return datetime_str

In [3]:
#filter by News
news = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//div[@class="filter_item" and @data-filter-value="News"]/a[@onclick="searchPage.dofacetedsearch(0,\'section\',\'News\')"]')))
driver.execute_script("arguments[0].click();", news)     
    
for i in range (0, 141):
    soup = get_soup(driver)
    get_articles(soup)
    try:
        link = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//a[@class='next_btn']")))
        driver.execute_script("arguments[0].click();", link) 
    except:
        break
    
df = pd.DataFrame(articlelist)
print(df)

Total articles to process: 20
Extracting articles 1 ...
Extracting articles 2 ...
Extracting articles 3 ...
Extracting articles 4 ...
Extracting articles 5 ...
Extracting articles 6 ...
Extracting articles 7 ...
Extracting articles 8 ...
Extracting articles 9 ...
Extracting articles 10 ...
Extracting articles 11 ...
Extracting articles 12 ...
Extracting articles 13 ...
Extracting articles 14 ...
Extracting articles 15 ...
Extracting articles 16 ...
Extracting articles 17 ...
Extracting articles 18 ...
Extracting articles 19 ...
Extracting articles 20 ...
Total articles to process: 20
Extracting articles 1 ...
Extracting articles 2 ...
Extracting articles 3 ...
Extracting articles 4 ...
Extracting articles 5 ...
Extracting articles 6 ...
Extracting articles 7 ...
Extracting articles 8 ...
Extracting articles 9 ...
Extracting articles 10 ...
Extracting articles 11 ...
Extracting articles 12 ...
Extracting articles 13 ...
Extracting articles 14 ...
Extracting articles 15 ...
Extracting ar

Extracting articles 16 ...
Extracting articles 17 ...
Extracting articles 18 ...
Extracting articles 19 ...
Extracting articles 20 ...
Total articles to process: 20
Extracting articles 1 ...
Extracting articles 2 ...
Extracting articles 3 ...
Extracting articles 4 ...
Extracting articles 5 ...
Extracting articles 6 ...
Extracting articles 7 ...
Extracting articles 8 ...
Extracting articles 9 ...
Extracting articles 10 ...
Extracting articles 11 ...
Extracting articles 12 ...
Extracting articles 13 ...
Extracting articles 14 ...
Extracting articles 15 ...
Extracting articles 16 ...
Extracting articles 17 ...
Extracting articles 18 ...
Extracting articles 19 ...
Extracting articles 20 ...
Total articles to process: 20
Extracting articles 1 ...
Extracting articles 2 ...
Extracting articles 3 ...
Extracting articles 4 ...
Extracting articles 5 ...
Extracting articles 6 ...
Extracting articles 7 ...
Extracting articles 8 ...
Extracting articles 9 ...
Extracting articles 10 ...
Extracting ar

Extracting articles 17 ...
Extracting articles 18 ...
Extracting articles 19 ...
Extracting articles 20 ...
Total articles to process: 20
Extracting articles 1 ...
Extracting articles 2 ...
Extracting articles 3 ...
Extracting articles 4 ...
Extracting articles 5 ...
Extracting articles 6 ...
Extracting articles 7 ...
Extracting articles 8 ...
Extracting articles 9 ...
Extracting articles 10 ...
Extracting articles 11 ...
Extracting articles 12 ...
Extracting articles 13 ...
Extracting articles 14 ...
Extracting articles 15 ...
Extracting articles 16 ...
Extracting articles 17 ...
Extracting articles 18 ...
Extracting articles 19 ...
Extracting articles 20 ...
Total articles to process: 20
Extracting articles 1 ...
Extracting articles 2 ...
Extracting articles 3 ...
Extracting articles 4 ...
Extracting articles 5 ...
Extracting articles 6 ...
Extracting articles 7 ...
Extracting articles 8 ...
Extracting articles 9 ...
Extracting articles 10 ...
Extracting articles 11 ...
Extracting ar

In [4]:
df = pd.DataFrame(articlelist)
df.to_csv(f'bursa_news_articles.csv', index=False)